In [1]:
def sudoku_reference():
    
    def cross(str_a, str_b):
        return [a + b for a in str_a for b in str_b]
    
    all_rows = 'ABCDEFGHI'
    all_columns = '123456789'
    
    coords = cross(all_rows, all_columns)
    
    row_units = [cross(r, all_columns) for r in all_rows]
    
    col_units = [cross(all_rows, c) for c in all_columns]
    
    box_units = [cross(r_sq, c_sq) for r_sq in ['ABC', 'DEF', 'GHI'] for c_sq in ['123', '456', '789']]
    
    all_units = row_units + col_units + box_units  
    
    groups = {}
    
    groups['units'] = {pos : [unit for unit in all_units if pos in unit] for pos in coords}
    
    groups['peers'] = {pos: set(sum(groups['units'][pos], [])) - {pos} for pos in coords}
    
    return coords, groups, all_units
    

In [2]:
def parse_puzzle(puzzle, digits = '123456789', nulls = '0.'):
    
    flat_puzzle = ['.' if char in nulls else char for char in puzzle if char in digits + nulls]
    coords, groups, all_units = sudoku_reference()
    return dict(zip(coords, flat_puzzle))

In [47]:
def solve_puzzle(puzzle):
    """Solves a Sudoku puzzle from a string input."""
    digits = '123456789'  # Using a string here instead of a list

    coords, groups, all_units = sudoku_reference()
    input_grid = parse_puzzle(puzzle)
    input_grid = {k: v for k, v in input_grid.items() if v != '.'}  # Filter so we only have confirmed cells
    output_grid = {cell: digits for cell in coords}  # Create a board where all digits are possible in each cell

    def confirm_value(grid, pos, val):
        """Confirms a value by eliminating all other remaining possibilities."""
        remaining_values = grid[pos].replace(val, '')  # Possibilities we can eliminate due to the confirmation
        for val in remaining_values:
            grid = eliminate(grid, pos, val)
#             print(grid)
        return grid

    def eliminate(grid, pos, val):
        """Eliminates `val` as a possibility from all peers of `pos`."""

        if grid is None:  
            return None

        if val not in grid[pos]:  # If we have already eliminated this value we can exit
            return grid

        grid[pos] = grid[pos].replace(val, '')  # Remove the possibility from the given cell

        if len(grid[pos]) == 0:  # If there are no remaining possibilities, we have made the wrong decision
            return None
        elif len(grid[pos]) == 1:  # We have confirmed the digit and so can remove that value from all peers now
            for peer in groups['peers'][pos]:
                grid = eliminate(grid, peer, grid[pos])  # Recurses, propagating the constraint
                if grid is None:  # Exit if grid has already found a contradiction
                    return None

        # Check for the number of remaining places the eliminated digit could possibly occupy
        for unit in groups['units'][pos]:
            possibilities = [p for p in unit if val in grid[p]]

            if len(possibilities) == 0:  # If there are no possible locations for the digit, we have made a mistake
                return None
            # If there is only one possible position and that still has multiple possibilities, confirm the digit
            elif len(possibilities) == 1 and len(grid[possibilities[0]]) > 1:
                if confirm_value(grid, possibilities[0], val) is None:
                    return None

        return grid


    for position, value in input_grid.items():  # For each value we're given, confirm the value
        output_grid = confirm_value(output_grid, position, value)

    return output_grid

In [48]:
puzzle =  "105070400080200000724001006000325000237000145600417000800100624000003050001040309"
         

In [50]:
values = solve_puzzle(puzzle)

In [66]:
print("Solved Sudoku\n")
for i in range(9):
    ch = chr(65 + i)
    for j in range(9):
        print(values[ch+str(j+1)],end = " ")
    print('\n')

Solved Sudoku

1 9 5 6 7 8 4 3 2 

3 8 6 2 9 4 5 1 7 

7 2 4 5 3 1 9 8 6 

4 1 9 3 2 5 7 6 8 

2 3 7 9 8 6 1 4 5 

6 5 8 4 1 7 2 9 3 

8 7 3 1 5 9 6 2 4 

9 4 2 7 6 3 8 5 1 

5 6 1 8 4 2 3 7 9 

